In [1]:
pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bayesian-optimization

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lightgbm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-optimize==0.8.1

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn==0.23.2

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pickle
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from utils import  *
from skopt import BayesSearchCV
%matplotlib inline
plt.style.use('fivethirtyeight')

## Part 2: Supervised Learning Model


## Load and clean the train and test dataset

In [7]:
mailout_train = pd.read_csv('../../data/mail_train.csv',sep=';')
mailout_test = pd.read_csv('../../data/mail_test.csv',sep=';')


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#process load data
attributes = pd.read_excel('../../data/attributes.xlsx', engine='openpyxl', skiprows = 1)

#train data
mailout_train = unknown_unify(mailout_train, attributes)
X = feature_transform(mailout_train)

#test data
mailout_test = unknown_unify(mailout_test, attributes)
X_sub = feature_transform(mailout_test)

#----!

y = X['RESPONSE']
X.drop(['RESPONSE'], axis = 1, inplace = True)

print(X.shape, X_sub.shape)

Creating PRAEGENDE_JUGENDJAHRE_DECADE feature
Creating PRAEGENDE_JUGENDJAHRE_MOVEMENT feature
Creating PRAEGENDE_JUGENDJAHRE_DECADE feature
Creating PRAEGENDE_JUGENDJAHRE_MOVEMENT feature
(42962, 364) (42833, 364)


## bayesian with simple tunning test

In [12]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

pbounds = {
    'learning_rate': (0.00, 1.0),
    'n_estimators': (100, 110),
    'max_depth': (3,10),
    'gamma': (0, 1),
    'subsample':(0.5,0.9),
    'colsample':(0.5,0.9)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        gamma,
                       subsample,
                       colsample):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)

    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma,
        subsample=subsample,
        colsample=colsample,
        eta=0.1,
        eval_metric='auc')
    return np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc'))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1
   
)

In [13]:
%%time
#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an acquisition function of expected improvement
optimizer.maximize(n_iter=5, init_points=10, acq='ei')

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:48:07] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:48:11] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




...

[09:56:09] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:56:23] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:56:37] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


|  15       |  0.6169   |  0.8667   |  0.09461  |  0.9866   |  9.629    |  100.1    |  0.6816   |
CPU times: user 1h 8min 11s, sys: 7.33 s, total: 1h 8min 18s
Wall time: 8min 44s


In [14]:
#Extracting the best parameters
params=optimizer.max['params']
print(params)

{'colsample': 0.8505556609184153, 'gamma': 0.8946066635038473, 'learning_rate': 0.08504421136977791, 'max_depth': 3.2733834826301766, 'n_estimators': 101.69830419564569, 'subsample': 0.8512570013717653}


In [15]:
#Converting the max_depth and n_estimator values from float to int
params['max_depth']= int(params['max_depth'])
params['n_estimators']=int(params['n_estimators'])

In [17]:
num_features = 366

#get the feature importances DataFrame and plot the barchart
model_fi = XGBClassifier( verbose = False).fit( X, y)
fi = pd.DataFrame({'features':X.columns, 'importance' : model_fi.feature_importances_/ model_fi.feature_importances_.sum()}).sort_values(['importance'], ascending = False)


#reduce the ftrain and test data based on the important features
X_reduced = X[fi['features'].head(num_features).to_list()]
X_sub_reduced = X_sub[fi['features'].head(num_features).to_list()]

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:57:24] WARNING: ../src/learner.cc:573: 
Parameters: { "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:57:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [18]:
#use the dictionaty with optimized parameters result of the BayesianOptimization
model = XGBClassifier(**params)

#predefined funciton for trainings and exporting the final model
predict_sub1(X_reduced, y, model, X_sub_reduced,mailout_test,'submission.csv')


[09:57:38] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:57:39] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
